# HW2
## 資料來源 https://www.kaggle.com/jsphyg/weather-dataset-rattle-package
## 目標 ：預測明天是否會下雨 ?
## 屬性 ：最高溫，最低溫，雨量，風向，風速，濕度，氣壓，今天是否有雲，今天是否下雨


## 規則 ：
### 當天降雨量 > 0，會下雨
### 當天有下雨，隔天會下雨
### 下午三點氣溫大於 15 度，會下雨

In [2]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.externals.six import StringIO   
from sklearn.tree import export_graphviz
import pydotplus

In [3]:
df = pd.read_csv('weatherAUS.csv')
df.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RISK_MM,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,0.0,No
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,0.0,No
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,0.0,No
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,1.0,No
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,0.2,No


In [4]:
df['RainTomorrow'].value_counts()

No     110316
Yes     31877
Name: RainTomorrow, dtype: int64

In [5]:
rain = {'No':0,'Yes':1}
df['RainToday'] = df['RainToday'].map(rain)
df['RainTomorrow'] = df['RainTomorrow'].map(rain)

In [6]:
df = df.drop('Date',axis=1)
df = df.drop('Location',axis=1)
df = df.drop('Evaporation',axis=1)
df = df.drop('Sunshine',axis=1)
df = df.drop('WindGustDir',axis=1)
df = df.drop('WindDir9am',axis=1)
df = df.drop('WindDir3pm',axis=1)
df = df.drop('Cloud9am',axis=1)
df = df.drop('Cloud3pm',axis=1)
df = df.drop('RISK_MM',axis=1)

In [7]:
df.isnull().sum()

MinTemp            637
MaxTemp            322
Rainfall          1406
WindGustSpeed     9270
WindSpeed9am      1348
WindSpeed3pm      2630
Humidity9am       1774
Humidity3pm       3610
Pressure9am      14014
Pressure3pm      13981
Temp9am            904
Temp3pm           2726
RainToday         1406
RainTomorrow         0
dtype: int64

### 清除 inf 與 nan

In [8]:
df = df.replace([np.inf, -np.inf], np.nan)
df = df.dropna()

In [9]:
df.isnull().sum()

MinTemp          0
MaxTemp          0
Rainfall         0
WindGustSpeed    0
WindSpeed9am     0
WindSpeed3pm     0
Humidity9am      0
Humidity3pm      0
Pressure9am      0
Pressure3pm      0
Temp9am          0
Temp3pm          0
RainToday        0
RainTomorrow     0
dtype: int64

### 取前 7000 筆資料訓練

In [10]:
df_train = df[:7000]
df_train.head()

,MinTemp,MaxTemp,Rainfall,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,13.4,22.9,0.6,44.0,20.0,24.0,71.0,22.0,1007.7,1007.1,16.9,21.8,0.0,0
1,7.4,25.1,0.0,44.0,4.0,22.0,44.0,25.0,1010.6,1007.8,17.2,24.3,0.0,0
2,12.9,25.7,0.0,46.0,19.0,26.0,38.0,30.0,1007.6,1008.7,21.0,23.2,0.0,0
3,9.2,28.0,0.0,24.0,11.0,9.0,45.0,16.0,1017.6,1012.8,18.1,26.5,0.0,0
4,17.5,32.3,1.0,41.0,7.0,20.0,82.0,33.0,1010.8,1006.0,17.8,29.7,0.0,0


### 用剩下資料測試

In [11]:
df_test = df[7000:]
df_test.head()

,MinTemp,MaxTemp,Rainfall,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
7289,15.7,33.7,0.0,41.0,24.0,24.0,18.0,5.0,1019.8,1015.6,22.7,33.6,0.0,0
7290,15.2,29.0,0.0,41.0,22.0,9.0,40.0,15.0,1019.1,1017.3,17.1,27.3,0.0,0
7291,10.3,32.5,0.0,33.0,15.0,17.0,20.0,11.0,1018.9,1015.6,19.8,30.3,0.0,0
7292,19.7,37.4,0.0,57.0,20.0,24.0,14.0,7.0,1015.3,1010.1,24.6,36.1,0.0,0
7293,17.1,32.8,0.0,39.0,15.0,20.0,30.0,8.0,1014.4,1013.9,22.5,31.7,0.0,0


In [12]:
y = df_train['RainTomorrow'].values
df_train = df_train.drop('RainTomorrow',1)
y

array([0, 0, 0, ..., 0, 0, 0])

### 建立 decesion tree

In [13]:
dtree=DecisionTreeClassifier(max_depth=4)
dtree.fit(df_train,y)

dot_data = StringIO()
export_graphviz(dtree, 
                out_file=dot_data,  
                filled=True, 
                feature_names=list(df_train),
                class_names=['rain','not rain'],
                special_characters=True)

graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
graph.write_pdf("tree.pdf")
graph.write_png("tree.png")

True

### 產生的 decesion tree
![tree](tree.png)

In [14]:
y_test = df_test['RainTomorrow'].values
X_test = df_test.drop('RainTomorrow', 1)

y_predict = dtree.predict(X_test)

y_predict

array([0, 0, 0, ..., 0, 0, 0])

In [15]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test, y_predict)

0.811404209965361

## 討論
### 決策樹產生的結果與一開始的規則不同
### 前一天是否下雨以及前一天的雨量並不會影響隔天是否下雨
### 影響隔天是否下雨最大的因素是當天下午三點的濕度
### 風速，氣壓，溫度也會產生影響